In [ ]:
# Installs
!pip install PyPDF2
!pip install requests
!pip install BeautifulSoup
!pip install tika
!pip install nltk
!pip install tabula-py
!java -version

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp37-none-any.whl size=61085 sha256=937804fad2ede325208a396df0e15e3f9c5b1ea369f2eb494bbe0c282d8cdd9f
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=5ec26f746120ceee13976be11dee39068589c0315259b397772ed6d1401a5cfe
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika
     |████████████████████████████████| 11.7MB 305kB/s 
openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [ ]:
# Imports
from PyPDF2 import PdfFileReader
import requests
import pandas as pd
import io
import re
from bs4 import BeautifulSoup 
from google.colab import files

In [ ]:
#Natural Language Processing Packages
import nltk

## Download Resources
nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("punkt")

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords, state_union
from nltk.tag import PerceptronTagger
import nltk.data
import tabula

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
#Keyword Dictionary
keyword_df = pd.read_excel("Keywords.xlsx")
keywords = {} 
keywords["Character"] = keyword_df['CHARACTER'].dropna().tolist()
keywords["Skating"] = keyword_df['SKATING'].dropna().tolist()
keywords["Puck Skill"] = keyword_df['PUCK SKILL'].dropna().tolist()
keywords["Shooting Skill"] = keyword_df['SHOOTING'].dropna().tolist()
unassigned = []

In [ ]:
print(keywords["Character"])
print(keywords["Skating"])
print(keywords["Puck Skill"])
print(keywords["Shooting Skill"])

Draft Analyst 2020 Draft Class

In [ ]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

source2020DA = 'DraftAnalyst 2020.pdf'
pdfFileObj = open(source2020DA, 'rb')
pdfReader = PdfFileReader(pdfFileObj)
tabula.convert_into(source2020DA,"DraftAnalyst 2020.csv", pages = "3-14") #Download and clean CSV.

uploaded = files.upload()
with open("DraftAnalyst 2020.csv") as csvfile:
  df = pd.read_csv(io.BytesIO(uploaded['DraftAnalyst 2020.csv'])) #Upload cleaned CSV.
  df.insert(2,column='SOURCE', value=source2020DA)
  df.insert(3,column='CHARACTER', value='')
  df.insert(4,column='SKATING', value='')
  df.insert(5,column='PUCK SKILL', value='')
  df.insert(6,column='SHOOTING SKILL', value='')

Saving DraftAnalyst 2020.csv to DraftAnalyst 2020 (1).csv


In [ ]:
df = df[0:101]
print (df)

     RANK                NAME  ... PUCK SKILL SHOOTING SKILL
0       1  Lafreniere, Alexis  ...                          
1       2    Byfield, Quinton  ...                          
2       3        Stutzle, Tim  ...                          
3       4      Perfetti, Cole  ...                          
4       5        Rossi, Marco  ...                          
..    ...                 ...  ...        ...            ...
96     97      Tyutnev, Pavel  ...                          
97     98    Didkovskiy, Ivan  ...                          
98     99      Savoie, Carter  ...                          
99    100      Ekmark, Elliot  ...                          
100   101        Coe, Brandon  ...                          

[101 rows x 7 columns]


In [ ]:
pageIndex = 0

pdfFileObj = open(source2020DA, 'rb')
pdfReader = PdfFileReader(pdfFileObj)
for page in range(13,114): # Page with Alex Lafreniere to page with Brandon Coe
  pageObj = pdfReader.getPage(page)  
  content = pageObj.extractText()
  # Clean the text
  cleaned = content.replace('\n','')
  cleaned = cleaned.replace('!', '.')
  cleaned = cleaned.replace('?', '.')
  cleaned = cleaned[cleaned.find('Scouting Report') + len('Scouting Report'):]
  cleaned = cleaned.replace('Scouting Report ', '')
  if (content.find('Bottom Line') != -1):
    cleaned = cleaned.replace('Bottom Line ', '')
  # Separate into phrases
  completed = ('\n'.join(sent_detector.tokenize(cleaned.strip())))
  phrase_list = completed.split('\n')
  # Categorize based on keywords
  character_phrases = []
  shooting_phrases = []
  skating_phrases = []
  puck_skill_phrases = []
  for phrase in phrase_list:
    if any(word in phrase for word in keywords['Character']):
      character_phrases.append(phrase)
    if any(word in phrase for word in keywords['Shooting Skill']):
      shooting_phrases.append(phrase)
    if any(word in phrase for word in keywords['Skating']):
      skating_phrases.append(phrase)
    if any(word in phrase for word in keywords['Puck Skill']):
      puck_skill_phrases.append(phrase)
    if not (any(word in phrase for word in keywords['Character']) or any(word in phrase for word in keywords['Shooting Skill']) or any(word in phrase for word in keywords['Skating']) or any(word in phrase for word in keywords['Puck Skill'])): # Temp check
      unassigned.append(phrase)
  # Add to Dataframe
  df.at[pageIndex,'CHARACTER'] = character_phrases
  df.at[pageIndex,'SHOOTING'] = shooting_phrases
  df.at[pageIndex,'SKATING'] = skating_phrases
  df.at[pageIndex,'PUCK SKILL'] = puck_skill_phrases
  pageIndex += 1
pdfFileObj.close()

print(df)

     RANK  ...                                           SHOOTING
0       1  ...  From his deadly wrist shot to his superior pla...
1       2  ...  [Byfield, however, is exceptionally crafty wit...
2       3  ...  [His quickness on or off the puck is evident f...
3       4  ...  [A lethal scorer who possesses an arsenal of d...
4       5  ...  [One of the more exciting forward prospects in...
..    ...  ...                                                ...
96     97  ...  [With 13 goals on only 53 shots for a 24.5 sho...
97     98  ...  [Not only is Didkovskiy a lethal goal scorer b...
98     99  ...  [The leading goal scorer in the AJHL, Savoie, ...
99    100  ...  [He ended up finishing as one of the top score...
100   101  ...  [He can score with an impressive shot-release ...

[101 rows x 8 columns]


In [ ]:
final_df = df[['NAME','SOURCE','CHARACTER','SKATING','PUCK SKILL','SHOOTING']]
final_df.to_csv("Draft Analyst 2020 Final.csv")

In [ ]:
for phrase in unassigned:
  print (phrase)
if unassigned == []:
  print ("empty")